# <p><center> Overview about the Pneumonia</center> </p>

![d](https://www.mayoclinic.org/-/media/kcms/gbs/patient-consumer/images/2016/05/18/13/02/ww5r032t-8col-jpg.jpg)

   Pneumonia is an infection that inflames the air sacs in one or both lungs. The air sacs may fill with fluid or pus (purulent material), causing cough with phlegm or pus, fever, chills, and difficulty breathing. A variety of organisms, including bacteria, viruses and fungi, can cause pneumonia. Pneumonia can range in seriousness from mild to life-threatening. It is most serious for infants and young children, people older than age 65, and people with health problems or weakened immune systems.
   Symptoms
The signs and symptoms of pneumonia vary from mild to severe, depending on factors such as the type of germ causing the infection, and your age and overall health. Mild signs and symptoms often are similar to those of a cold or flu, but they last longer.

#### Signs and symptoms of pneumonia may include:

*  Chest pain when you breathe or cough
* Confusion or changes in mental awareness (in adults age 65 and older)
* Cough, which may produce phlegm
* Fatigue
* Fever, sweating and shaking chills
* Lower than normal body temperature (in adults older than age 65 and people with weak immune systems)
* Nausea, vomiting or diarrhea
* Shortness of breath

Therefore, in this project, we used the evidence that is in front of you to help discover this disease as soon as possible so that it is not severely affected by the patient, and we hope that we have done what we can.
   

# Importing Libraries

In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

# load The image

In [ ]:
train_dir = "../input/chest-xray-pneumonia/chest_xray/train"
test_dir = "../input/chest-xray-pneumonia/chest_xray/test"
val_dir = "../input/chest-xray-pneumonia/chest_xray/val"

### Show the size of image

In [ ]:
num_pneumonia = len(os.listdir(os.path.join(train_dir,'PNEUMONIA')))
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))


In [ ]:
print(f"PNEUMONIA={num_pneumonia}")
print(f"NORMAL={num_normal}")

In [ ]:
print(f"PNEUMONIA={len(os.listdir(os.path.join(test_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(test_dir, 'NORMAL')))}")

In [ ]:
print(f"PNEUMONIA={len(os.listdir(os.path.join(val_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(val_dir, 'NORMAL')))}")

In [ ]:
pneumonia = os.listdir("../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA")
pneumonia_dir = "../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA"

In [ ]:
# Train Normal 
NORMAL = os.listdir("../input/chest-xray-pneumonia/chest_xray/train/NORMAL")
NORMAL_dir = "../input/chest-xray-pneumonia/chest_xray/train/NORMAL"

In [ ]:
# Show Normal image
plt.figure(figsize=(15, 5))

for i in range(12):
    plt.subplot(4, 3, i + 1)
    img = plt.imread(os.path.join(NORMAL_dir, NORMAL[i]))
    plt.title("NORMAL")
    plt.imshow(img, cmap='gray')
    plt.axis('on')
    

plt.tight_layout()

In [ ]:
# shaoe PNEUMONIA image
plt.figure(figsize=(15, 5))

for i in range(12):
    plt.subplot(4, 3, i + 1)
    img = plt.imread(os.path.join(pneumonia_dir, pneumonia[i]))
    plt.title("PNEUMONIA")
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    

plt.tight_layout()

## Creat a Label for the data

In [ ]:
import glob

pneumonia_train = glob.glob(train_dir+"/PNEUMONIA/*.jpeg")
normal_train = glob.glob(train_dir+"/NORMAL/*.jpeg")

In [ ]:
# Here we creat label 0 for (Normal) 1 for( pneumonia )
data = pd.DataFrame(np.concatenate([[0]*len(normal_train) ,
                                    [1]*len(pneumonia_train)]),columns=["class"])

In [ ]:
data.head(4000)

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data['class'],data=data,palette='rocket',color='red')
plt.title('PNEUMONIA vs NORMAL')
plt.show()

# Here we can Augmented Data

In [ ]:
img_Datagen = ImageDataGenerator(
        rescale = 1/255,
        shear_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.5,2.0],
        width_shift_range = 0.2,
        rotation_range=20,
        fill_mode = 'nearest'
)
val_Datagen = ImageDataGenerator(
        rescale = 1/255
)

## For Train data

In [ ]:
train = img_Datagen.flow_from_directory(train_dir,
                                       batch_size=8,
                                       class_mode='binary',
#                                        target_size=(224,224,3))
                                       )

## For validation data

In [ ]:

validation = val_Datagen.flow_from_directory(val_dir,
                                              batch_size=8,
                                              class_mode='binary',
#                                               target_size=(224,224,3))
                                            )

## For test data

In [ ]:
test = val_Datagen.flow_from_directory(test_dir,
                                       batch_size=8,
                                       class_mode='binary')

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data['class'],data=data,palette='rocket',color='red')
plt.title('PNEUMONIA vs NORMAL')
plt.show()
len(train)

In [ ]:
img, label = next(train)

In [ ]:
print("The shape of img:",img.shape)
print("The shape of label:",label.shape)

# Using First Model VGG19


In [ ]:
vgg_model = tf.keras.applications.VGG19(
    weights='imagenet',
    include_top = False,
#     input_shape = (224,224,3)
)

#!pip install git+https://github.com/qubvel/segmentation_models
#import efficientnet.keras as efn 

#modeleff = efn.EfficientNetB0(weights='imagenet',include_top=False,)

#for layer in modeleff.layers:
 #   layer.trainable=False
    
x = vgg_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
# output layer
predictions = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=vgg_model.input, outputs=predictions)

'TF_GPU_ALLOCATOR=cuda_malloc_async'

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models
    

In [ ]:
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

In [ ]:
vgg_model = tf.keras.applications.VGG19(
    weights='imagenet',
    include_top = False,
#     input_shape = (224,224,3)
)

from tensorflow.keras.applications import EfficientNetB0
modeleff = EfficientNetB0(weights='imagenet',include_top = False,)

for layer in modeleff.layers:
    layer.trainable=False
    
x = modeleff.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# output layer
predictions = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=modeleff.input, outputs=predictions)

In [ ]:
# to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=8)

In [ ]:
# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['Precision'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical, plot_model
plot_model(model, to_file='cnn-mnist.png', show_shapes=True)

In [ ]:
history = model.fit(train,epochs=30, 
                    validation_data=validation,
                     steps_per_epoch=50,
                    callbacks=[early_stopping,lr],
                    verbose=True,
                    batch_size=8)

# Cheak the Evaluate

In [ ]:
# Evaluating the model for  train and test
score = model.evaluate(train)

print("Train Loss is: ", score[0])
print("Train Accuracy is: ", score[1])

In [ ]:
# Test data
score = model.evaluate(test)

print("Test Loss is: ", score[0])
print("Test Accuracy is: ", score[1])

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('EVALUATION OF VGG19')

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
prediction1=model.predict(test)

In [ ]:
prediction1[:10]

# ResNet50V2

In [ ]:
resnet_model = tf.keras.applications.ResNet50V2(
    weights='imagenet',
    include_top = False,
   # input_shape = (224,224,3)
)

for layer in resnet_model.layers:
    layer.trainable=False
    
x = resnet_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
# output layer
predictions = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model2 = tf.keras.Model(inputs=resnet_model.input, outputs=predictions)



In [ ]:
model2.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical, plot_model
plot_model(model2, to_file='cnn-mnist.png', show_shapes=True)

In [ ]:
# Compiling the model
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['Precision'])

In [ ]:
from cosine_annealing import CosineAnnealingScheduler

callbacks_new = [
    CosineAnnealingScheduler(T_max=100, eta_max=1e-2, eta_min=1e-4)
]


In [ ]:
history = model2.fit(train,epochs=50, 
                    validation_data=validation,
                     steps_per_epoch=50,
                     callbacks=callbacks_new,
                    batch_size=8)

# Evaluating the model on train and test

In [ ]:
# Evaluating the model on train 
score = model2.evaluate(train)

print("Train Loss: ", score[0])
print("Train Accuracy: ", score[1])

In [ ]:
# Evaluating the model test
score = model2.evaluate(test)
print("\nTest loss: ", score[0])
print("Test Accuracy: ", score[1])

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
prediction2=model2.predict(test)


In [ ]:
prediction2[:10]